# Structural Data
As the structural values of the lsoa area, describe the background featuers of each lsoa.
- population density
- age groups (elderly)
- ethnic (minority)

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

## Population

Population density and population growth rate

In [37]:
# reading population density
density11_path = "data/stru_data/mid-2011-lsoa-pop-density.xls"
pop_den11 = pd.ExcelFile(density11_path)

In [38]:
# reading sheet
df_den_11 = pop_den11.parse(sheet_name="Mid-2011 Density") 
df_den_11['Mid-2011 population'] = df_den_11['People per Sq Km']*df_den_11['Area Sq Km']

In [39]:
# select columns
den11_col = ['Code', 'People per Sq Km', 'Mid-2011 population']
den_11 = df_den_11[den11_col]

In [40]:
# reading the density for 2015
density14_path = "data/stru_data/SAPE20DT11-mid-2014-lsoa-population-density.xls"
pop_den14 = pd.ExcelFile(density14_path)
df_den_14 = pop_den14.parse(sheet_name="Mid-2014 Population Density", header = 4) 
df_den_14.head()

,Code,Name,Mid-2014 population,Area Sq Km,People per Sq Km
0,E01020634,County Durham 001A,1535,6.4975,236
1,E01020635,County Durham 001B,1344,1.6963,792
2,E01020636,County Durham 001C,1724,0.9272,1859
3,E01020654,County Durham 001D,1871,0.9884,1893
4,E01020676,County Durham 001E,1470,5.4717,269


In [41]:
# select columns
den14_col = ['Code', 'Mid-2014 population', 'People per Sq Km']
den_14 = df_den_14[den14_col]

In [42]:
# join together
# 把 df2 的列名改成一致
den_14.rename(columns={'Code': 'LSOA Code'}, inplace=True)
den_11.rename(columns={'Code': 'LSOA Code'}, inplace=True)
den_14.rename(columns={'People per Sq Km': '2014_population_density'}, inplace=True)
den_11.rename(columns={'People per Sq Km': '2011_population_density'}, inplace=True)

# 然后 merge
den_all = den_11.merge(den_14, on='LSOA Code', how='left')
print(den_all.head())

   LSOA Code  2011_population_density  Mid-2011 population  \
0  E01020634               241.323586               1568.0   
1  E01020635               811.766787               1377.0   
2  E01020636              1827.006040               1694.0   
3  E01020654              1764.467827               1744.0   
4  E01020676               277.244732               1517.0   

   Mid-2014 population  2014_population_density  
0                 1535                      236  
1                 1344                      792  
2                 1724                     1859  
3                 1871                     1893  
4                 1470                      269  


C:\Users\hp\AppData\Local\Temp\ipykernel_29508\2470234484.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  den_14.rename(columns={'Code': 'LSOA Code'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\2470234484.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  den_11.rename(columns={'Code': 'LSOA Code'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\2470234484.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [43]:
den_all['pop_growth_rate'] = (den_all['Mid-2014 population'] - den_all['Mid-2011 population']) / den_all['Mid-2011 population']
den_all['avg_den'] = (den_all['2011_population_density'] + den_all['2014_population_density']) / 2
print(den_all.head())

   LSOA Code  2011_population_density  Mid-2011 population  \
0  E01020634               241.323586               1568.0   
1  E01020635               811.766787               1377.0   
2  E01020636              1827.006040               1694.0   
3  E01020654              1764.467827               1744.0   
4  E01020676               277.244732               1517.0   

   Mid-2014 population  2014_population_density  pop_growth_rate      avg_den  
0                 1535                      236        -0.021046   238.661793  
1                 1344                      792        -0.023965   801.883393  
2                 1724                     1859         0.017710  1843.003020  
3                 1871                     1893         0.072821  1828.733913  
4                 1470                      269        -0.030982   273.122366  


In [44]:
den_all.isna().sum()

LSOA Code                  0
2011_population_density    0
Mid-2011 population        0
Mid-2014 population        0
2014_population_density    0
pop_growth_rate            0
avg_den                    0
dtype: int64

In [45]:
# read in lsoa shapefile
london_gdf = gpd.read_file("data/statistical-gis-boundaries-london/statistical-gis-boundaries-london/ESRI/LSOA_2011_London_gen_MHW.shp")

# extract lsoa11cd as LSOA list
london_lsoa_list = london_gdf['LSOA11CD'].unique()

# select data in London area
df_london_den = den_all[den_all['LSOA Code'].isin(london_lsoa_list)]
df_london_den.shape

(4835, 7)

## Age Group

focusing on the elderly(over 65) percentage, as a description date.

In [29]:
# read in files
age12_path = "data/stru_data/SAPE20DT12-mid-2012-lsoa-Broad_ages-estimates-formatted.xls"
age12 = pd.ExcelFile(age12_path)
age_group_12 = age12.parse(sheet_name="Mid-2012 Persons", header = 4) 
# age_group_14 = age.parse(sheet_name="Mid-2014 LSOA 2021", header = 3) 

In [30]:
# select columns
age_col = ['Area Codes', '65+', 'All Ages']
age_12 = age_group_12[age_col]
age_12["elderly_per_12"] = age_12["65+"] * 100 / age_12["All Ages"]
print(age_12.columns)

Index(['Area Codes', '65+', 'All Ages', 'elderly_per_12'], dtype='object')


C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3066496738.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_12["elderly_per_12"] = age_12["65+"] * 100 / age_12["All Ages"]


In [ ]:
# select columns
age14_path = "data/stru_data/SAPE20DT12-mid-2014-lsoa-Broad_ages-estimates-formatted.xls"
age14 = pd.ExcelFile(age14_path)
age_group_14 = age14.parse(sheet_name="Mid-2014 Persons", header = 4) 
age_col = ['Area Codes', '65+', 'All Ages']
age_14 = age_group_14[age_col]
age_14["elderly_per_14"] = age_14["65+"] * 100 / age_14["All Ages"]
print(age_14.columns)

Index(['Area Codes', '65+', 'All Ages', 'elderly_per_14'], dtype='object')


C:\Users\hp\AppData\Local\Temp\ipykernel_29508\242502156.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_14["elderly_per_14"] = age_14["65+"] * 100 / age_14["All Ages"]


In [34]:
age_group = age_12[["Area Codes", "elderly_per_12"]].merge(age_14[["Area Codes", "elderly_per_14"]], on = "Area Codes", how = "left")

In [36]:
age_group["senior_per"] = (age_group["elderly_per_12"] + age_group["elderly_per_14"]) / 2
age_group.rename(columns={'Area Codes': 'LSOA Code'}, inplace=True)
print(age_group.head(5))

   LSOA Code  elderly_per_12  elderly_per_14  senior_per
0  E06000047       18.779180       19.604384   19.191782
1  E01020634       22.251309       23.192182   22.721746
2  E01020635       19.285714       20.907738   20.096726
3  E01020636       19.530792       19.257541   19.394166
4  E01020654       17.471264       21.485836   19.478550


In [46]:
# select data in London area
df_london_age = age_group[age_group['LSOA Code'].isin(london_lsoa_list)]
df_london_age.shape

(4835, 4)

## Ethnic groups

Focusing on the minority percentage here.

In [2]:
ethnic_group = pd.read_csv("data/stru_data/bulk.csv")
ethnic_group.head(5)

,date,geography,geography code,Ethnic Group: All categories: Ethnic group; measures: Value,Ethnic Group: White; measures: Value,Ethnic Group: Gypsy / Traveller / Irish Traveller; measures: Value,Ethnic Group: Mixed / Multiple ethnic group; measures: Value,Ethnic Group: Asian / Asian British: Indian; measures: Value,Ethnic Group: Asian / Asian British: Pakistani; measures: Value,Ethnic Group: Asian / Asian British: Bangladeshi; measures: Value,Ethnic Group: Asian / Asian British: Chinese; measures: Value,Ethnic Group: Asian / Asian British: Other Asian; measures: Value,Ethnic Group: Black / African / Caribbean / Black British; measures: Value,Ethnic Group: Other Ethnic Group; measures: Value
0,2011,Darlington 001B,E01012334,2466,2401,3,29,18,3,0,5,1,1,5
1,2011,Darlington 001C,E01012335,1383,1362,0,9,0,0,5,3,3,0,1
2,2011,Darlington 001D,E01012366,2008,1961,5,16,8,1,1,1,9,6,0
3,2011,Darlington 001E,E01033481,1364,1311,5,16,11,3,0,5,3,6,4
4,2011,Darlington 001F,E01033482,1621,1540,26,20,14,0,0,6,3,11,1


In [3]:
# select columns
eth_col = ['geography code', 'Ethnic Group: All categories: Ethnic group; measures: Value', 'Ethnic Group: White; measures: Value']
df_eth = ethnic_group[eth_col]

In [4]:
df_eth.rename(columns={'geography code': 'LSOA Code'}, inplace=True)
df_eth.rename(columns={'Ethnic Group: All categories: Ethnic group; measures: Value': 'all'}, inplace=True)
df_eth.rename(columns={'Ethnic Group: White; measures: Value': 'white'}, inplace=True)
df_eth[['all', 'white']].isnull().sum()

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\4047659333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eth.rename(columns={'geography code': 'LSOA Code'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\4047659333.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eth.rename(columns={'Ethnic Group: All categories: Ethnic group; measures: Value': 'all'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\4047659333.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

all      0
white    0
dtype: int64

In [5]:
df_eth['minority_per'] = (df_eth['all'] - df_eth['white']) *100 / df_eth['all']
print(df_eth.head())

   LSOA Code   all  white  minority_per
0  E01012334  2466   2401      2.635848
1  E01012335  1383   1362      1.518438
2  E01012366  2008   1961      2.340637
3  E01033481  1364   1311      3.885630
4  E01033482  1621   1540      4.996915


C:\Users\hp\AppData\Local\Temp\ipykernel_29508\191691982.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eth['minority_per'] = (df_eth['all'] - df_eth['white']) *100 / df_eth['all']


In [47]:
# select data in London area
df_london_eth = df_eth[df_eth['LSOA Code'].isin(london_lsoa_list)]
df_london_eth.shape

(4835, 4)

In [48]:
# merge all the data together
df_data_all = df_london_den.merge(df_london_age, on='LSOA Code', how='left')

In [49]:
df_data_all = df_data_all.merge(df_london_eth, on='LSOA Code', how='left')
print(df_data_all.head(5))

   LSOA Code  2011_population_density  Mid-2011 population  \
0  E01000907             13348.880597               1431.0   
1  E01000908              8937.142857               1564.0   
2  E01000909              7007.874016               1602.0   
3  E01000912              1773.239594               1589.0   
4  E01000913              1048.237477               1695.0   

   Mid-2014 population  2014_population_density  pop_growth_rate  \
0                 1443                    13461         0.008386   
1                 1662                     9497         0.062660   
2                 1671                     7310         0.043071   
3                 1711                     1909         0.076778   
4                 1818                     1124         0.072566   

        avg_den  elderly_per_12  elderly_per_14  senior_per   all  white  \
0  13404.940299       11.626298       12.404712   12.015505  1430   1051   
1   9217.071429       13.616490       14.981949   14.299220  1581 

## House and Dewlling

In [50]:
dwe = pd.read_csv("data/stru_data/dewlling_house.csv")
dwe.head(5)

,date,geography,geography code,Rural Urban,Dwelling Type: All categories: Dwelling type; measures: Value,Dwelling Type: Unshared dwelling; measures: Value,Dwelling Type: Shared dwelling: Two household spaces; measures: Value,Dwelling Type: Shared dwelling: Three or more household spaces; measures: Value,Dwelling Type: All categories: Household spaces; measures: Value,Dwelling Type: Household spaces with at least one usual resident; measures: Value,Dwelling Type: Household spaces with no usual residents; measures: Value,Dwelling Type: Whole house or bungalow: Detached; measures: Value,Dwelling Type: Whole house or bungalow: Semi-detached; measures: Value,Dwelling Type: Whole house or bungalow: Terraced (including end-terrace); measures: Value,"Dwelling Type: Flat, maisonette or apartment: Purpose-built block of flats or tenement; measures: Value","Dwelling Type: Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits); measures: Value","Dwelling Type: Flat, maisonette or apartment: In a commercial building; measures: Value",Dwelling Type: Caravan or other mobile or temporary structure; measures: Value
0,2011,Darlington 001B,E01012334,Total,1017,1016,0,1,1019,962,57,603,217,175,10,3,7,4
1,2011,Darlington 001C,E01012335,Total,635,635,0,0,635,620,15,256,201,153,19,1,5,0
2,2011,Darlington 001D,E01012366,Total,908,908,0,0,908,860,48,456,214,203,17,6,10,2
3,2011,Darlington 001E,E01033481,Total,588,588,0,0,588,554,34,152,215,135,79,1,5,1
4,2011,Darlington 001F,E01033482,Total,665,664,1,0,666,650,16,257,331,52,14,0,0,12


In [51]:
# select columns
dwe_col = ['geography code', 'Dwelling Type: All categories: Dwelling type; measures: Value', 
           'Dwelling Type: Unshared dwelling; measures: Value',
           'Dwelling Type: Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits); measures: Value']
df_dwe = dwe[dwe_col]

In [52]:
df_dwe.rename(columns={'geography code': 'LSOA Code'}, inplace=True)
df_dwe.rename(columns={'Dwelling Type: All categories: Dwelling type; measures: Value': 'all_dwelling'}, inplace=True)
df_dwe.rename(columns={'Dwelling Type: Unshared dwelling; measures: Value': 'unshared'}, inplace=True)
df_dwe.rename(columns={'Dwelling Type: Flat, maisonette or apartment: Part of a converted or shared house (including bed-sits); measures: Value': 'converted'}, inplace=True)
df_dwe[['all_dwelling', 'unshared', 'converted']].isnull().sum()

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3422320615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dwe.rename(columns={'geography code': 'LSOA Code'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3422320615.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dwe.rename(columns={'Dwelling Type: All categories: Dwelling type; measures: Value': 'all_dwelling'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3422320615.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

all_dwelling    0
unshared        0
converted       0
dtype: int64

In [53]:
df_dwe['shared_per'] = (df_dwe['all_dwelling'] - df_dwe['unshared']) *100 / df_dwe['all_dwelling']
df_dwe['converted_per'] = df_dwe['converted'] *100 / df_dwe['all_dwelling']
print(df_dwe.head())

   LSOA Code  all_dwelling  unshared  converted  shared_per  converted_per
0  E01012334          1017      1016          3    0.098328       0.294985
1  E01012335           635       635          1    0.000000       0.157480
2  E01012366           908       908          6    0.000000       0.660793
3  E01033481           588       588          1    0.000000       0.170068
4  E01033482           665       664          0    0.150376       0.000000


C:\Users\hp\AppData\Local\Temp\ipykernel_29508\355280031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dwe['shared_per'] = (df_dwe['all_dwelling'] - df_dwe['unshared']) *100 / df_dwe['all_dwelling']
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\355280031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dwe['converted_per'] = df_dwe['converted'] *100 / df_dwe['all_dwelling']


In [54]:
# select data in London area
df_london_dwe = df_dwe[df_dwe['LSOA Code'].isin(london_lsoa_list)]
df_london_dwe.shape

(4835, 6)

In [55]:
df_data_all = df_data_all.merge(df_london_dwe, on='LSOA Code', how='left')
print(df_data_all.head(5))

   LSOA Code  2011_population_density  Mid-2011 population  \
0  E01000907             13348.880597               1431.0   
1  E01000908              8937.142857               1564.0   
2  E01000909              7007.874016               1602.0   
3  E01000912              1773.239594               1589.0   
4  E01000913              1048.237477               1695.0   

   Mid-2014 population  2014_population_density  pop_growth_rate  \
0                 1443                    13461         0.008386   
1                 1662                     9497         0.062660   
2                 1671                     7310         0.043071   
3                 1711                     1909         0.076778   
4                 1818                     1124         0.072566   

        avg_den  elderly_per_12  elderly_per_14  senior_per   all  white  \
0  13404.940299       11.626298       12.404712   12.015505  1430   1051   
1   9217.071429       13.616490       14.981949   14.299220  1581 

## Education

In [56]:
edu = pd.read_csv("data/stru_data/education.csv", header = 6)
edu.head(5)

,2011 super output area - lower layer,All categories: Highest level of qualification,%,Highest level of qualification: Level 1 qualifications,%.1,Highest level of qualification: Level 2 qualifications,%.2,Highest level of qualification: Apprenticeship,%.3,Highest level of qualification: Level 3 qualifications,%.4,Highest level of qualification: Level 4 qualifications and above,%.5
0,E01000001 : City of London 001A,1350,100.0,35.0,2.6,75.0,5.6,7.0,0.5,94.0,7.0,1047.0,77.6
1,E01000002 : City of London 001B,1293,100.0,41.0,3.2,71.0,5.5,5.0,0.4,91.0,7.0,1024.0,79.2
2,E01000003 : City of London 001C,1242,100.0,81.0,6.5,111.0,8.9,19.0,1.5,93.0,7.5,706.0,56.8
3,E01000005 : City of London 001E,821,100.0,67.0,8.2,92.0,11.2,7.0,0.9,78.0,9.5,283.0,34.5
4,E01000006 : Barking and Dagenham 016A,1309,100.0,155.0,11.8,169.0,12.9,20.0,1.5,115.0,8.8,431.0,32.9


In [57]:
# select columns
edu['LSOA Code'] = edu['2011 super output area - lower layer'].str.split(':').str[0].str.strip()
edu_col = ['LSOA Code', 'Highest level of qualification: Level 4 qualifications and above', '%.5']
df_edu = edu[edu_col]

In [58]:
df_edu.rename(columns={'Highest level of qualification: Level 4 qualifications and above': 'level_4_pop'}, inplace=True)
df_edu.rename(columns={'%.5': 'level_4_per'}, inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\916735863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edu.rename(columns={'Highest level of qualification: Level 4 qualifications and above': 'level_4_pop'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\916735863.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edu.rename(columns={'%.5': 'level_4_per'}, inplace=True)


In [59]:
# select data in London area
df_london_edu = df_edu[df_edu['LSOA Code'].isin(london_lsoa_list)]
df_london_edu.shape

(4835, 3)

In [60]:
edu_numeric = df_london_edu.select_dtypes(include=['number'])
df_london_edu[edu_numeric.columns] = edu_numeric.fillna(0)

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\1901529358.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_london_edu[edu_numeric.columns] = edu_numeric.fillna(0)


In [61]:
print(df_london_edu)

       LSOA Code  level_4_pop  level_4_per
0      E01000001       1047.0         77.6
1      E01000002       1024.0         79.2
2      E01000003        706.0         56.8
3      E01000005        283.0         34.5
4      E01000006        431.0         32.9
...          ...          ...          ...
32817  E01033742        532.0         48.3
32818  E01033743        533.0         40.9
32819  E01033744        404.0         29.1
32820  E01033745        463.0         38.0
32821  E01033746        713.0         56.8

[4835 rows x 3 columns]


In [62]:
df_data_all = df_data_all.merge(df_london_edu, on='LSOA Code', how='left')
print(df_data_all.columns)

Index(['LSOA Code', '2011_population_density', 'Mid-2011 population',
       'Mid-2014 population', '2014_population_density', 'pop_growth_rate',
       'avg_den', 'elderly_per_12', 'elderly_per_14', 'senior_per', 'all',
       'white', 'minority_per', 'all_dwelling', 'unshared', 'converted',
       'shared_per', 'converted_per', 'level_4_pop', 'level_4_per'],
      dtype='object')


## Occupation

In [63]:
occ = pd.read_csv("data/stru_data/occupation.csv", header = 7)
occ.head(5)

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\1107269988.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  occ = pd.read_csv("data/stru_data/occupation.csv", header = 7)


,2011 super output area - lower layer,All categories: Occupation,%,"1. Managers, directors and senior officials",%.1,2. Professional occupations,%.2,3. Associate professional and technical occupations,%.3,4. Administrative and secretarial occupations,...,5. Skilled trades occupations,%.5,"6. Caring, leisure and other service occupations",%.6,7. Sales and customer service occupations,%.7,8. Process plant and machine operatives,%.8,9. Elementary occupations,%.9
0,E01000001 : City of London 001A,926,100.0,167.0,18.0,446.0,48.2,208.0,22.5,53.0,...,15.0,1.6,17.0,1.8,6.0,0.6,2.0,0.2,12.0,1.3
1,E01000002 : City of London 001B,910,100.0,171.0,18.8,417.0,45.8,207.0,22.7,61.0,...,12.0,1.3,8.0,0.9,18.0,2.0,3.0,0.3,13.0,1.4
2,E01000003 : City of London 001C,764,100.0,89.0,11.6,285.0,37.3,183.0,24.0,67.0,...,19.0,2.5,36.0,4.7,29.0,3.8,11.0,1.4,45.0,5.9
3,E01000005 : City of London 001E,476,100.0,49.0,10.3,81.0,17.0,90.0,18.9,64.0,...,31.0,6.5,33.0,6.9,33.0,6.9,26.0,5.5,69.0,14.5
4,E01000006 : Barking and Dagenham 016A,781,100.0,60.0,7.7,140.0,17.9,77.0,9.9,81.0,...,103.0,13.2,73.0,9.3,70.0,9.0,73.0,9.3,104.0,13.3


In [64]:
# select columns
occ['LSOA Code'] = occ['2011 super output area - lower layer'].str.split(':').str[0].str.strip()
occ_col = ['LSOA Code', '2. Professional occupations', '%.2']
df_occ = occ[occ_col]

In [65]:
df_occ.rename(columns={'2. Professional occupations': 'prof_occ_count'}, inplace=True)
df_occ.rename(columns={'%.2': 'prof_occ_per'}, inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3747434857.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_occ.rename(columns={'2. Professional occupations': 'prof_occ_count'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3747434857.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_occ.rename(columns={'%.2': 'prof_occ_per'}, inplace=True)


In [66]:
# select data in London area
df_london_occ = df_occ[df_occ['LSOA Code'].isin(london_lsoa_list)]
df_london_occ.shape

(4835, 3)

In [67]:
occ_numeric = df_london_occ.select_dtypes(include=['number'])
df_london_occ[occ_numeric.columns] = occ_numeric.fillna(0)

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\1206008326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_london_occ[occ_numeric.columns] = occ_numeric.fillna(0)


In [68]:
print(df_london_occ)

       LSOA Code  prof_occ_count  prof_occ_per
0      E01000001           446.0          48.2
1      E01000002           417.0          45.8
2      E01000003           285.0          37.3
3      E01000005            81.0          17.0
4      E01000006           140.0          17.9
...          ...             ...           ...
32817  E01033742           204.0          26.7
32818  E01033743           180.0          22.0
32819  E01033744           120.0          15.5
32820  E01033745           146.0          18.5
32821  E01033746           299.0          37.9

[4835 rows x 3 columns]


In [69]:
df_data_all = df_data_all.merge(df_london_occ, on='LSOA Code', how='left')
print(df_data_all.columns)

Index(['LSOA Code', '2011_population_density', 'Mid-2011 population',
       'Mid-2014 population', '2014_population_density', 'pop_growth_rate',
       'avg_den', 'elderly_per_12', 'elderly_per_14', 'senior_per', 'all',
       'white', 'minority_per', 'all_dwelling', 'unshared', 'converted',
       'shared_per', 'converted_per', 'level_4_pop', 'level_4_per',
       'prof_occ_count', 'prof_occ_per'],
      dtype='object')


## Property Sales

In [82]:
# reading house price
pro_path = "data/stru_data/HPSSA Dataset 41 - Number of residential property sales by LSOA.xls"
pro_price = pd.ExcelFile(pro_path)
# reading sheet
df_pro = pro_price.parse(sheet_name="Data", header = 5) 
df_pro.head()

,Local authority code,Local authority name,LSOA code,LSOA name,Year ending Dec 1995,Year ending Mar 1996,Year ending Jun 1996,Year ending Sep 1996,Year ending Dec 1996,Year ending Mar 1997,...,Year ending Dec 2020,Year ending Mar 2021,Year ending Jun 2021,Year ending Sep 2021,Year ending Dec 2021,Year ending Mar 2022,Year ending Jun 2022,Year ending Sep 2022,Year ending Dec 2022,Year ending Mar 2023
0,E06000001,Hartlepool,E01011949,Hartlepool 009A,12,13,14,15,15,15,...,17,14,22,25,25,28,23,24,24,20
1,E06000001,Hartlepool,E01011950,Hartlepool 008A,44,45,45,46,43,51,...,36,36,38,45,50,49,51,41,35,33
2,E06000001,Hartlepool,E01011951,Hartlepool 007A,43,33,34,43,47,51,...,26,33,37,34,33,33,40,44,40,39
3,E06000001,Hartlepool,E01011952,Hartlepool 002A,24,22,18,16,15,19,...,5,4,10,12,13,15,11,9,7,5
4,E06000001,Hartlepool,E01011953,Hartlepool 002B,8,9,7,10,9,8,...,4,3,3,5,6,5,10,13,14,15


In [83]:
df_pro['pro_sale_2010'] = df_pro['Year ending Mar 2010'] + df_pro['Year ending Jun 2010'] + df_pro['Year ending Sep 2010'] + df_pro['Year ending Dec 2010']
df_pro['pro_sale_2015'] = df_pro['Year ending Mar 2015'] + df_pro['Year ending Jun 2015'] + df_pro['Year ending Sep 2015'] + df_pro['Year ending Dec 2015']
df_pro['sale_gro'] = (df_pro['pro_sale_2015'] - df_pro['pro_sale_2010'])
df_pro['sale_avg'] = (df_pro['pro_sale_2015'] + df_pro['pro_sale_2010']) /2

print(df_pro.head())

  Local authority code Local authority name  LSOA code        LSOA name  \
0            E06000001           Hartlepool  E01011949  Hartlepool 009A   
1            E06000001           Hartlepool  E01011950  Hartlepool 008A   
2            E06000001           Hartlepool  E01011951  Hartlepool 007A   
3            E06000001           Hartlepool  E01011952  Hartlepool 002A   
4            E06000001           Hartlepool  E01011953  Hartlepool 002B   

   Year ending Dec 1995  Year ending Mar 1996  Year ending Jun 1996  \
0                    12                    13                    14   
1                    44                    45                    45   
2                    43                    33                    34   
3                    24                    22                    18   
4                     8                     9                     7   

   Year ending Sep 1996  Year ending Dec 1996  Year ending Mar 1997  ...  \
0                    15                    15 

In [84]:
df_pro.isna().sum()

Local authority code    0
Local authority name    0
LSOA code               0
LSOA name               0
Year ending Dec 1995    0
                       ..
Year ending Mar 2023    0
pro_sale_2010           0
pro_sale_2015           0
sale_gro                0
sale_avg                0
Length: 118, dtype: int64

In [86]:
pro_col = ['LSOA code', 'sale_avg', 'sale_gro']
df_pro = df_pro[pro_col]
print(df_pro.head())

   LSOA code  sale_avg  sale_gro
0  E01011949      59.0         6
1  E01011950      59.0        48
2  E01011951      72.5       -19
3  E01011952      32.0        16
4  E01011953      28.5         3


In [87]:
df_pro.rename(columns={'LSOA code': 'LSOA Code'}, inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3836450377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pro.rename(columns={'LSOA code': 'LSOA Code'}, inplace=True)


In [88]:
df_data_all = df_data_all.merge(df_pro, on='LSOA Code', how='left')
print(df_data_all.columns)

Index(['LSOA Code', '2011_population_density', 'Mid-2011 population',
       'Mid-2014 population', '2014_population_density', 'pop_growth_rate',
       'avg_den', 'elderly_per_12', 'elderly_per_14', 'senior_per', 'all',
       'white', 'minority_per', 'all_dwelling', 'unshared', 'converted',
       'shared_per', 'converted_per', 'level_4_pop', 'level_4_per',
       'prof_occ_count', 'prof_occ_per', 'sale_avg', 'sale_gro'],
      dtype='object')


## Number of bedrooms

In [89]:
room = pd.read_csv("data/stru_data/rooms.csv", header = 6)
room.head(5)

,2011 super output area - lower layer,All categories: Number of rooms,1 room,2 rooms,3 rooms,4 rooms,5 rooms,6 rooms,7 rooms,8 rooms,9 or more rooms
0,E01000001 : City of London 001A,876,92.0,87.0,278.0,181.0,120.0,71.0,34.0,10.0,3.0
1,E01000002 : City of London 001B,830,10.0,136.0,266.0,233.0,64.0,60.0,34.0,18.0,9.0
2,E01000003 : City of London 001C,817,128.0,196.0,231.0,212.0,44.0,3.0,2.0,0.0,1.0
3,E01000005 : City of London 001E,467,43.0,89.0,142.0,109.0,68.0,12.0,3.0,0.0,1.0
4,E01000006 : Barking and Dagenham 016A,543,8.0,17.0,82.0,59.0,145.0,140.0,63.0,23.0,6.0


In [90]:
# select columns
room['LSOA Code'] = room['2011 super output area - lower layer'].str.split(':').str[0].str.strip()
room_col = ['LSOA Code', 'All categories: Number of rooms', '1 room', '2 rooms']
df_room = room[room_col]

In [91]:
# select data in London area
df_london_room = df_room[df_room['LSOA Code'].isin(london_lsoa_list)]
df_london_room.shape

(4835, 4)

In [92]:
print(df_london_room.dtypes)   # 确认数据类型
print(df_london_room.isna().sum())  # 检查缺失值

LSOA Code                           object
All categories: Number of rooms     object
1 room                             float64
2 rooms                            float64
dtype: object
LSOA Code                          0
All categories: Number of rooms    0
1 room                             0
2 rooms                            0
dtype: int64


In [93]:
df_london_room.rename(columns={'All categories: Number of rooms': 'all_household'}, inplace=True)
cols_to_convert = ['1 room', '2 rooms', 'all_household']  # 你用到的列

for col in cols_to_convert:
    df_london_room[col] = pd.to_numeric(df_london_room[col], errors='coerce')

C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3764146254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_london_room.rename(columns={'All categories: Number of rooms': 'all_household'}, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_29508\3764146254.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_london_room[col] = pd.to_numeric(df_london_room[col], errors='coerce')


In [94]:
df_london_room['less_2_rooms_per'] = (df_london_room['1 room'] + df_london_room['2 rooms'])*100 / df_london_room['all_household']
print(df_london_room.isna().sum())  # 检查缺失值

LSOA Code           0
all_household       0
1 room              0
2 rooms             0
less_2_rooms_per    0
dtype: int64


C:\Users\hp\AppData\Local\Temp\ipykernel_29508\2827628396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_london_room['less_2_rooms_per'] = (df_london_room['1 room'] + df_london_room['2 rooms'])*100 / df_london_room['all_household']


In [95]:
print(df_london_room.head())

   LSOA Code  all_household  1 room  2 rooms  less_2_rooms_per
0  E01000001            876    92.0     87.0         20.433790
1  E01000002            830    10.0    136.0         17.590361
2  E01000003            817   128.0    196.0         39.657283
3  E01000005            467    43.0     89.0         28.265525
4  E01000006            543     8.0     17.0          4.604052


In [96]:
df_data_all = df_data_all.merge(df_london_room, on='LSOA Code', how='left')
print(df_data_all.columns)

Index(['LSOA Code', '2011_population_density', 'Mid-2011 population',
       'Mid-2014 population', '2014_population_density', 'pop_growth_rate',
       'avg_den', 'elderly_per_12', 'elderly_per_14', 'senior_per', 'all',
       'white', 'minority_per', 'all_dwelling', 'unshared', 'converted',
       'shared_per', 'converted_per', 'level_4_pop', 'level_4_per',
       'prof_occ_count', 'prof_occ_per', 'sale_avg', 'sale_gro',
       'all_household', '1 room', '2 rooms', 'less_2_rooms_per'],
      dtype='object')


## Marriage

In [97]:
single = pd.read_csv("data/stru_data/marriage.csv", header = 7)
single.head(5)

,2011 super output area - lower layer,All categories: Marital and civil partnership status,Single (never married or never registered a same-sex civil partnership)
0,E01000001 : City of London 001A,876,374.0
1,E01000002 : City of London 001B,830,327.0
2,E01000003 : City of London 001C,817,442.0
3,E01000005 : City of London 001E,467,217.0
4,E01000006 : Barking and Dagenham 016A,543,127.0


In [98]:
# select columns
single['LSOA Code'] = single['2011 super output area - lower layer'].str.split(':').str[0].str.strip()
single.rename(columns={'All categories: Marital and civil partnership status': 'all_resident'}, inplace=True)
single.rename(columns={'Single (never married or never registered a same-sex civil partnership)': 'single'}, inplace=True)
print(single.dtypes)   # 确认数据类型


2011 super output area - lower layer     object
all_resident                             object
single                                  float64
LSOA Code                                object
dtype: object


In [99]:
# select data in London area
single = single[single['LSOA Code'].isin(london_lsoa_list)]
single.shape

(4835, 4)

In [100]:
cols_to_convert_sin = ['all_resident', 'single']  # 你用到的列

for col in cols_to_convert_sin:
    single[col] = pd.to_numeric(single[col], errors='coerce')

In [101]:
single['single_per'] = single['single']*100/single['all_resident']
print(single.head(5))

    2011 super output area - lower layer  all_resident  single  LSOA Code  \
0        E01000001 : City of London 001A           876   374.0  E01000001   
1        E01000002 : City of London 001B           830   327.0  E01000002   
2        E01000003 : City of London 001C           817   442.0  E01000003   
3        E01000005 : City of London 001E           467   217.0  E01000005   
4  E01000006 : Barking and Dagenham 016A           543   127.0  E01000006   

   single_per  
0   42.694064  
1   39.397590  
2   54.100367  
3   46.466809  
4   23.388582  


In [102]:
sin_col = ['LSOA Code', 'single_per']
df_single = single[sin_col]

In [103]:
df_data_all = df_data_all.merge(df_single, on='LSOA Code', how='left')
print(df_data_all.columns)

Index(['LSOA Code', '2011_population_density', 'Mid-2011 population',
       'Mid-2014 population', '2014_population_density', 'pop_growth_rate',
       'avg_den', 'elderly_per_12', 'elderly_per_14', 'senior_per', 'all',
       'white', 'minority_per', 'all_dwelling', 'unshared', 'converted',
       'shared_per', 'converted_per', 'level_4_pop', 'level_4_per',
       'prof_occ_count', 'prof_occ_per', 'sale_avg', 'sale_gro',
       'all_household', '1 room', '2 rooms', 'less_2_rooms_per', 'single_per'],
      dtype='object')


## Distance travelled to work

In [104]:
travel_dis = pd.read_csv("data/stru_data/distance_to_work.csv", header = 6)
travel_dis.head(5)

,2011 super output area - lower layer,All categories: Distance travelled to work,Less than 10km,10km to less than 30km,30km and over,Work mainly at or from home,Other
0,E01000001 : City of London 001A,937,695.0,26.0,21.0,146.0,49.0
1,E01000002 : City of London 001B,919,660.0,27.0,20.0,155.0,57.0
2,E01000003 : City of London 001C,769,582.0,30.0,18.0,82.0,57.0
3,E01000005 : City of London 001E,481,345.0,37.0,18.0,36.0,45.0
4,E01000006 : Barking and Dagenham 016A,781,297.0,258.0,36.0,61.0,129.0


In [105]:
# select columns
travel_dis['LSOA Code'] = travel_dis['2011 super output area - lower layer'].str.split(':').str[0].str.strip()
travel_dis.rename(columns={'All categories: Distance travelled to work': 'all_resident'}, inplace=True)
travel_dis.rename(columns={'Less than 10km': 'less_10km'}, inplace=True)
print(travel_dis.dtypes)   # 确认数据类型

2011 super output area - lower layer     object
all_resident                             object
less_10km                               float64
10km to less than 30km                  float64
30km and over                           float64
Work mainly at or from home             float64
Other                                   float64
LSOA Code                                object
dtype: object


In [106]:
# select data in London area
travel_dis = travel_dis[travel_dis['LSOA Code'].isin(london_lsoa_list)]
travel_dis.shape

(4835, 8)

In [107]:
cols_to_convert_dis = ['all_resident', 'less_10km']  # 你用到的列

for col in cols_to_convert_dis:
    travel_dis[col] = pd.to_numeric(travel_dis[col], errors='coerce')

In [108]:
travel_dis['less_10km_per'] = travel_dis['less_10km']*100/travel_dis['all_resident']
print(travel_dis.head(5))

    2011 super output area - lower layer  all_resident  less_10km  \
0        E01000001 : City of London 001A           937      695.0   
1        E01000002 : City of London 001B           919      660.0   
2        E01000003 : City of London 001C           769      582.0   
3        E01000005 : City of London 001E           481      345.0   
4  E01000006 : Barking and Dagenham 016A           781      297.0   

   10km to less than 30km  30km and over  Work mainly at or from home  Other  \
0                    26.0           21.0                        146.0   49.0   
1                    27.0           20.0                        155.0   57.0   
2                    30.0           18.0                         82.0   57.0   
3                    37.0           18.0                         36.0   45.0   
4                   258.0           36.0                         61.0  129.0   

   LSOA Code  less_10km_per  
0  E01000001      74.172892  
1  E01000002      71.817193  
2  E01000003  

In [109]:
travel_dis_col = ['LSOA Code', 'less_10km_per']
df_travel_dis = travel_dis[travel_dis_col]

In [110]:
df_data_all = df_data_all.merge(df_travel_dis, on='LSOA Code', how='left')
print(df_data_all.columns)

Index(['LSOA Code', '2011_population_density', 'Mid-2011 population',
       'Mid-2014 population', '2014_population_density', 'pop_growth_rate',
       'avg_den', 'elderly_per_12', 'elderly_per_14', 'senior_per', 'all',
       'white', 'minority_per', 'all_dwelling', 'unshared', 'converted',
       'shared_per', 'converted_per', 'level_4_pop', 'level_4_per',
       'prof_occ_count', 'prof_occ_per', 'sale_avg', 'sale_gro',
       'all_household', '1 room', '2 rooms', 'less_2_rooms_per', 'single_per',
       'less_10km_per'],
      dtype='object')


## Save Copy

In [113]:
columns_to_keep = ['LSOA Code', 'pop_growth_rate', 'avg_den', 'senior_per', 'minority_per', 
                   'all_dwelling', 'shared_per', 'converted_per',
                   'level_4_pop', 'level_4_per',
                   'prof_occ_count', 'prof_occ_per', 'sale_avg',
                   'single_per',
                   'less_10km_per']
cleaned_df = df_data_all[columns_to_keep]

In [114]:
cleaned_df.to_csv("data/all_stru_data_10_14.csv", index=False)